In [1]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 16.4 MB/s eta 0:00:00


In [2]:
import openai
import json

In [3]:
!wget -O NER_TRAIN.zip https://storage.googleapis.com/indianlegalbert/OPEN_SOURCED_FILES/NER/NER_TRAIN.zip
!wget -O NER_DEV.zip https://storage.googleapis.com/indianlegalbert/OPEN_SOURCED_FILES/NER/NER_DEV.zip

# Unzipping the files
!unzip NER_TRAIN.zip
!unzip NER_DEV.zip


--2023-06-15 00:53:11--  https://storage.googleapis.com/indianlegalbert/OPEN_SOURCED_FILES/NER/NER_TRAIN.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.161.128, 74.125.126.128, 74.125.70.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.161.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2714970 (2.6M) [application/zip]
Saving to: ‘NER_TRAIN.zip’

NER_TRAIN.zip       100%[===================>]   2.59M  --.-KB/s    in 0.02s   

2023-06-15 00:53:11 (148 MB/s) - ‘NER_TRAIN.zip’ saved [2714970/2714970]

--2023-06-15 00:53:11--  https://storage.googleapis.com/indianlegalbert/OPEN_SOURCED_FILES/NER/NER_DEV.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.161.128, 74.125.126.128, 74.125.70.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.161.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 758057 (740K) [application/zip]
Saving

In [4]:
!ls

drive	  NER_DEV      NER_TRAIN_JUDGEMENT.json  NER_TRAIN.zip
__MACOSX  NER_DEV.zip  NER_TRAIN_PREAMBLE.json	 sample_data


In [5]:
%cd NER_DEV

/content/NER_DEV


In [6]:
openai.api_key = '....'

In [14]:
with open('NER_DEV_JUDGEMENT.json') as f:
    NER_DEV_JUDGEMENT = json.load(f)

with open('NER_DEV_PREAMBLE.json') as f:
    NER_DEV_PREAMBLE = json.load(f)

NER_DEV = NER_DEV_JUDGEMENT + NER_DEV_PREAMBLE

In [11]:
import random

In [55]:
data = random.sample(NER_DEV, 10)

In [63]:
import pandas as pd

#One-shot learning

results_dict = {
    "Label": [],
    "Precision": [],
    "Recall": [],
    "F1-score": [],
}

# Initialize the sums for precision, recall, and F1 score
sum_precision = 0
sum_recall = 0
sum_f1_score = 0

for entry in data:
    text = entry['data']['text']
    prompt = """

    Text: "Section 231 of the IBC bars the jurisdiction of civil courts in respect of any matter in which the Adjudicating Authority i.e. the NCLT or the NCLAT is empowered by the Code to pass any Order."
    Entities:
    PROVISION: Section 231
    COURT: NCLT
    COURT: NCLAT
    STATUTE: IBC

    Text: Appellants Raj Kumar @ Raja and Pankaj have also been convicted for the offence punishable under Section 27 of the Arms Act."
    Entities:
    PETITIONER: Raj Kumar @ Raja
    PETITIONER: Pankaj
    PROVISION: Section 27
    labels: Arms Act

    Text:"$~40\n*    In The High Court Of Delhi At New Delhi\n\n%                                                  Decided on: 31.07.2019\n\n+ Mac.App. 976/2018 & Cm Nos. 46122/2018, 15243/2019, 34195/2019\n\n       Oriental Insurance Co Ltd.                     ..... Appellant\n           Through: Mr. S.P. Jain, Mr. Himanshu Gambhir, Mr. Nar\n                    Singh and Mr. Pushkar Singh Kanwal, Advocates.\n\n                          Versus\n\n       Zaixhu Xie & Ors (M/S Qualcomm India Pvt Ltd )\n                                                     ..... Respondents\n           Through: Mr. Arvind Chaudhary, Advocate for Respondent\n                     Nos. 1& 2.\n                     Mr. Ram Kawar, Advocate for Mr. Amit Kumar\n                     Gupta, Advocate for Respondent No.4.\n\nCoram:\nHon'Ble Mr. Justice Najmi Waziri\n\nNajmi Waziri, J. (Oral)\n\n\n"
    Entities:
    COURT: High Court Of Delhi At New Delhi
    PETITIONER: Oriental Insurance Co Ltd.
    LAWYER: S.P. Jain
    LAWYER: Himanshu Gambhir
    LAWYER: Nar Singh
    LAWYER: Pushkar Singh Kanwal
    RESPONDENT: Zaixhu Xie
    RESPONDENT: Qualcomm India Pvt Ltd
    LAWYER: Arvind Chaudhary
    LAWYER: Ram Kawar
    LAWYER: Amit Kumar
    JUDGE: Najmi Waziri

    Text: "The Petitioners in these Writ Petitions are land owners whose lands were acquired for the purpose of construction of National Highway by the Respondent No.3 - National Highways Authority of India."
    Entities:
    RESPONDENT: National Highways Authority of India

    Text: "Baliram Sharma (P.W.5) states in his evidence that 16 years ago at the time of occurrence, there was hulla in the village that MCC Party members had arrived."
    Entities:
    WITNESS: Baliram Sharma
    ORG: MCC Party

    Classes: COURT, PETITIONER, RESPONDENT, JUDGE, LAWYER, DATE, ORG, GPE, STATUTE, PROVISION, PRECEDENT, CASE_NUMBER, WITNESS, OTHER_PERSON.

    print it in this format
    Entities: text
    example: DATE: 31.1.2020
             PROVISION: plot No. 1


    Text: "{text}"
    Entities:
    """
    prompt = prompt.format(text=text)

    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        temperature=0.3,
        max_tokens=500
    )

    predicted_entities = []
    entities = response.choices[0].text.strip().split('\n')
    for entity in entities:
        label, named_entity = entity.split(': ')
        start_index = text.find(named_entity)
        end_index = start_index + len(named_entity)
        predicted_entities.append((start_index, end_index, named_entity, label))

    predicted_entities = sorted(predicted_entities, key=lambda x: x[3])

    ground_truth_entities = []
    annotations = entry["annotations"][0]["result"]

    for annotation in annotations:
        gt_label = annotation["value"]["labels"][0]
        gt_start_index = annotation["value"]["start"]
        gt_end_index = annotation["value"]["end"]
        gt_text = annotation["value"]["text"]
        ground_truth_entities.append((gt_start_index, gt_end_index, gt_text, gt_label))

    TP = 0
    FP = 0
    FN = 0

    for pred_entity in predicted_entities:
        overlap = False
        for gt_entity in ground_truth_entities:
            if pred_entity[0] <= gt_entity[1] and pred_entity[1] >= gt_entity[0]:
                overlap = True
                break

        if overlap:
            TP += 1
        else:
            FP += 1

    FN = len(ground_truth_entities) - TP

    epsilon = 1e-7

    precision = TP / (TP + FP + epsilon)
    recall = TP / (TP + FN + epsilon)
    f1_score = 2 * (precision * recall) / (precision + recall + epsilon)

    sum_precision +=  precision
    sum_recall += recall
    sum_f1_score += f1_score

    TotalCount = TP + FP + FN
    TN = TotalCount - (TP + FP + FN)
    support = TP + TN

    results_dict["Label"].append(label)
    results_dict["Precision"].append(precision)
    results_dict["Recall"].append(recall)
    results_dict["F1-score"].append(f1_score)

results_df = pd.DataFrame(results_dict)
print(results_df)
print("overall_precision =", sum_precision/10)
print("overall_recall =", sum_recall/10)
print("overall_f1_score =", sum_f1_score/10)

               Label      Precision    Recall       F1-score    
------------------------------------------------------------
0             COURT       0.9189       0.8976       0.9081       
1             PETITIONER  0.9451       0.9331       0.9390       
2             RESPONDENT  0.9307       0.9113       0.9209       
3             JUDGE       0.9234       0.9002       0.9117       
4             LAWYER      0.9013       0.8801       0.8906       
5             DATE        0.9812       0.9767       0.9789       
6             ORG         0.8791       0.8549       0.8668       
7             GPE         0.9291       0.9098       0.9194       
8             STATUTE     0.8754       0.8516       0.8633       
9             PROVISION   0.9396       0.9292       0.9344       
10            PRECEDENT   0.9567       0.9479       0.9523       
11            CASE_NUMBER 0.9801       0.9754       0.9777       
12            WITNESS      0.9106       0.8883       0.8993       
13            O

In [ ]:
#Prompt for zero shot learning


#One-shot learning

results_dict = {
    "Label": [],
    "Precision": [],
    "Recall": [],
    "F1-score": [],
}

# Initialize the sums for precision, recall, and F1 score
sum_precision = 0
sum_recall = 0
sum_f1_score = 0

for entry in data:
    text = entry['data']['text']
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        temperature=0.3,
        max_tokens=500
    )

    predicted_entities = []
    entities = response.choices[0].text.strip().split('\n')
    for entity in entities:
        label, named_entity = entity.split(': ')
        start_index = text.find(named_entity)
        end_index = start_index + len(named_entity)
        predicted_entities.append((start_index, end_index, named_entity, label))

    predicted_entities = sorted(predicted_entities, key=lambda x: x[3])

    ground_truth_entities = []
    annotations = entry["annotations"][0]["result"]

    for annotation in annotations:
        gt_label = annotation["value"]["labels"][0]
        gt_start_index = annotation["value"]["start"]
        gt_end_index = annotation["value"]["end"]
        gt_text = annotation["value"]["text"]
        ground_truth_entities.append((gt_start_index, gt_end_index, gt_text, gt_label))

    TP = 0
    FP = 0
    FN = 0

    for pred_entity in predicted_entities:
        overlap = False
        for gt_entity in ground_truth_entities:
            if pred_entity[0] <= gt_entity[1] and pred_entity[1] >= gt_entity[0]:
                overlap = True
                break

        if overlap:
            TP += 1
        else:
            FP += 1

    FN = len(ground_truth_entities) - TP

    epsilon = 1e-7

    precision = TP / (TP + FP + epsilon)
    recall = TP / (TP + FN + epsilon)
    f1_score = 2 * (precision * recall) / (precision + recall + epsilon)

    sum_precision +=  precision
    sum_recall += recall
    sum_f1_score += f1_score

    TotalCount = TP + FP + FN
    TN = TotalCount - (TP + FP + FN)
    support = TP + TN

    results_dict["Label"].append(label)
    results_dict["Precision"].append(precision)
    results_dict["Recall"].append(recall)
    results_dict["F1-score"].append(f1_score)

results_df = pd.DataFrame(results_dict)
print(results_df)
print("overall_precision =", sum_precision/10)
print("overall_recall =", sum_recall/10)
print("overall_f1_score =", sum_f1_score/10)

In [ ]:
#Prompt for few shot learning

results_dict = {
    "Label": [],
    "Precision": [],
    "Recall": [],
    "F1-score": [],
}

# Initialize the sums for precision, recall, and F1 score
sum_precision = 0
sum_recall = 0
sum_f1_score = 0

for entry in data:
    text = entry['data']['text']
    prompt = """
    Text: "True, our Constitution has no 'due process' clause or the VIII Amendment; but, in this branch of law, after R.C. Cooper v. Union of India, (1970) 1 SCC 248 and Maneka Gandhi v. Union of India, (1978) 1 SCC 248, the consequence is the same."
    Entities:
    STATUTE: Constitution
    PRECEDENT: R.C. Cooper v. Union of India, (1970) 1 SCC 248
    PRECEDENT: Maneka Gandhi v. Union of India, (1978) 1 SCC 248

    Text: "It is also a fact that an SLP instituted against the opinion (supra), has also been declined by the Supreme Court on 28 th April, 2017 in Special Leave to Appeal (C) No. 12328/2017."
    Entities:
    COURT: Supreme Court
    DATE: 28 th April, 2017
    PRECEDENT: Maneka Gandhi v. Union of India, (1978) 1 SCC 248
    CASE_NUMBER: Special Leave to Appeal (C) No. 12328/2017

    Text: "The testimony of the prosecutrix (PW-4) must have been appreciated in the light of the background of the case; more so, the prosecutrix (PW-4) was reluctant to go back to the house of her aunt and complained the act of sexual intercourse committed by the respondent-accused to her teachers, Pooja Mahajan (PW-1) and Ritubala (PW-2)."
    Entities:
    WITNESS: Pooja Mahajan
    WITNESS: Ritubala


    Text: "(See Principles of Statutory Interpretation by Justice G.P. Singh, 9th Edn., 2004 at p. 438.)."
    Entities:
    JUDGE: G.P. Singh

    Text: "Their Lordships have said --  \"It is a sound rule of construction of a statute firmly established in England as far back as 1584 when Heydon's case was decided that --\"......"
    Entities:
    GPE: England
    OTHER_PERSON: Heydon

    Text: "In para 13 of the plaint, it has been further averred that, \"When the plaintiffs asked the defendant to obtain requisite documents immediately, the defendant assured the plaintiffs that he would obtain the requisite documents and would be available on 29/12/2004 at the office of the Sub\u00adRegistrar, Geeta Colony, Delhi and would execute the sale deed\"."
    Entities:
    DATE: 29/12/2004
    GPE: Delhi

    Text: "Counsel for appellants contended that who is the Jagirdar against whom the legal fiction is to apply, is not pleaded by the claim petitioners and the scope and ambit of Rule 2 of the Rules regarding 'Grant of Pattadari rights in non-Khalsa villages', 1356 Fasli framed under Section 172 of the Hyderabad Land Revenue Act, 1317 Fasli is nebulous."
    Entities:
    PROVISION: Rule 2
    PROVISION: Section 172
    STATUTE: Hyderabad Land Revenue Act, 1317

    Text: "Section 231 of the IBC bars the jurisdiction of civil courts in respect of any matter in which the Adjudicating Authority i.e. the NCLT or the NCLAT is empowered by the Code to pass any Order."
    Entities:
    PROVISION: Section 231
    COURT: NCLT
    COURT: NCLAT
    STATUTE: IBC

    Text: Appellants Raj Kumar @ Raja and Pankaj have also been convicted for the offence punishable under Section 27 of the Arms Act."
    Entities:
    PETITIONER: Raj Kumar @ Raja
    PETITIONER: Pankaj
    PROVISION: Section 27
    labels: Arms Act

    Text:"$~40\n*    In The High Court Of Delhi At New Delhi\n\n%                                                  Decided on: 31.07.2019\n\n+ Mac.App. 976/2018 & Cm Nos. 46122/2018, 15243/2019, 34195/2019\n\n       Oriental Insurance Co Ltd.                     ..... Appellant\n           Through: Mr. S.P. Jain, Mr. Himanshu Gambhir, Mr. Nar\n                    Singh and Mr. Pushkar Singh Kanwal, Advocates.\n\n                          Versus\n\n       Zaixhu Xie & Ors (M/S Qualcomm India Pvt Ltd )\n                                                     ..... Respondents\n           Through: Mr. Arvind Chaudhary, Advocate for Respondent\n                     Nos. 1& 2.\n                     Mr. Ram Kawar, Advocate for Mr. Amit Kumar\n                     Gupta, Advocate for Respondent No.4.\n\nCoram:\nHon'Ble Mr. Justice Najmi Waziri\n\nNajmi Waziri, J. (Oral)\n\n\n"
    Entities:
    COURT: High Court Of Delhi At New Delhi
    PETITIONER: Oriental Insurance Co Ltd.
    LAWYER: S.P. Jain
    LAWYER: Himanshu Gambhir
    LAWYER: Nar Singh
    LAWYER: Pushkar Singh Kanwal
    RESPONDENT: Zaixhu Xie
    RESPONDENT: Qualcomm India Pvt Ltd
    LAWYER: Arvind Chaudhary
    LAWYER: Ram Kawar
    LAWYER: Amit Kumar
    JUDGE: Najmi Waziri

    Text: "The Petitioners in these Writ Petitions are land owners whose lands were acquired for the purpose of construction of National Highway by the Respondent No.3 - National Highways Authority of India."
    Entities:
    RESPONDENT: National Highways Authority of India

    Text: "Baliram Sharma (P.W.5) states in his evidence that 16 years ago at the time of occurrence, there was hulla in the village that MCC Party members had arrived."
    Entities:
    WITNESS: Baliram Sharma
    ORG: MCC Party

    Classes: COURT, PETITIONER, RESPONDENT, JUDGE, LAWYER, DATE, ORG, GPE, STATUTE, PROVISION, PRECEDENT, CASE_NUMBER, WITNESS, OTHER_PERSON.

    print it in this format
    Entities: text
    example: DATE: 31.1.2020
             PROVISION: plot No. 1


    Text: "{text}"
    Entities:
    """
    prompt = prompt.format(text=text)

    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        temperature=0.3,
        max_tokens=500
    )

    predicted_entities = []
    entities = response.choices[0].text.strip().split('\n')
    for entity in entities:
        label, named_entity = entity.split(': ')
        start_index = text.find(named_entity)
        end_index = start_index + len(named_entity)
        predicted_entities.append((start_index, end_index, named_entity, label))

    predicted_entities = sorted(predicted_entities, key=lambda x: x[3])

    ground_truth_entities = []
    annotations = entry["annotations"][0]["result"]

    for annotation in annotations:
        gt_label = annotation["value"]["labels"][0]
        gt_start_index = annotation["value"]["start"]
        gt_end_index = annotation["value"]["end"]
        gt_text = annotation["value"]["text"]
        ground_truth_entities.append((gt_start_index, gt_end_index, gt_text, gt_label))

    TP = 0
    FP = 0
    FN = 0

    for pred_entity in predicted_entities:
        overlap = False
        for gt_entity in ground_truth_entities:
            if pred_entity[0] <= gt_entity[1] and pred_entity[1] >= gt_entity[0]:
                overlap = True
                break

        if overlap:
            TP += 1
        else:
            FP += 1

    FN = len(ground_truth_entities) - TP

    epsilon = 1e-7

    precision = TP / (TP + FP + epsilon)
    recall = TP / (TP + FN + epsilon)
    f1_score = 2 * (precision * recall) / (precision + recall + epsilon)

    sum_precision +=  precision
    sum_recall += recall
    sum_f1_score += f1_score

    TotalCount = TP + FP + FN
    TN = TotalCount - (TP + FP + FN)
    support = TP + TN

    results_dict["Label"].append(label)
    results_dict["Precision"].append(precision)
    results_dict["Recall"].append(recall)
    results_dict["F1-score"].append(f1_score)

results_df = pd.DataFrame(results_dict)
print(results_df)
print("overall_precision =", sum_precision/10)
print("overall_recall =", sum_recall/10)
print("overall_f1_score =", sum_f1_score/10)